# Clasificación de Textos - Yelp! 

## Introducción

En este Notebook se realiza una clasificación de textos del dataset de [Yelp!](https://www.yelp.com/dataset).  
De este dataset, se utiliza el fichero *review.json* el cual contiene el conjunto de opiniones de los distintos clientes y el número de "estrellas" asignados. El objetivo es a partir del texto poder predecir el número de estrellas a asignar, lo cual corresponde a una clasificación basada en el sentimiento.

## Desarrollo

### Librerías

En primer lugar se importan las librerías necesarias para la realización de este trabajo:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import logging
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt

### Pre-processing

Se realiza le pre-procesamiento de la información, el cual está dividido en:
1. Lectura del dataset
2. Selección de variables
3. Eliminación de catacteres especiales y paso a *lowercase*

In [ ]:
root_path = 'yelp_dataset/review.json'

start = time.time()

data = pd.read_json(root_path_windows, lines=True)
data = data[["text", "stars"]]
data["text"] = data["text"].str.replace('[^a-zA-Z]',' ').str.lower()

end = time.time()
print("Seconds: ", end - start)

Además, se realiza un split en datos de entrenamiento y datos de test (80% - 20%)

In [ ]:
start = time.time()
x_train, x_test, y_train, y_test = train_test_split(data['text'],data['stars'], test_size = 0.2)
end = time.time()
print("Seconds: ", end - start)

#### Muestreo
Debido a que se va a analizar el comportamiento de los algoritmos con el aumento del tamaño muestral, también se realiza un meustreo de los datos leidos a aprtir de la selección de un porcentaje del dataset. Para ello se seleccionan los siguientes porcentajes: 10%, 30%, 50%, 75%.

In [ ]:
start = time.time()
data2 = data.head(int(len(data)*0.1))
x_train_10, x_test_10, y_train_10, y_test_10 = train_test_split(data2['text'],data2['stars'], test_size = 0.2)
end = time.time()
print("Seconds: ", end - start)

In [ ]:
start = time.time()
data3 = data.head(int(len(data)*0.3))
x_train_30, x_test_30, y_train_30, y_test_30 = train_test_split(data3['text'],data3['stars'], test_size = 0.2)
end = time.time()
print("Seconds: ", end - start)

In [ ]:
start = time.time()
data5 = data.head(int(len(data)*0.5))
x_train_50, x_test_50, y_train_50, y_test_50 = train_test_split(data5['text'],data5['stars'], test_size = 0.2)
end = time.time()
print("Seconds: ", end - start)

In [ ]:
start = time.time()
data75 = data.head(int(len(data)*0.75))
x_train_75, x_test_75, y_train_75, y_test_75 = train_test_split(data75['text'],data75['stars'], test_size = 0.2)
end = time.time()
print("Seconds: ", end - start)

In [ ]:
start = time.time()
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(x_train_10["data"])
end = time.time()
print("Seconds: ", end - start)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
start = time.time()
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(x_train_10)
end = time.time()
print("Seconds: ", end - start)

### Vectorización del texto
En el siguiente apartado se procede a extraer las *features* a partir del texto. Para ello se usan los algoritmos de CountVectorizer y TfidfVectorizer. Si bien estos algoritmos luego se incluyen en el Pipeline definido para el entrenamiento de los modelos estadísticos, este paso se realiza para obtener los tiempos arrojados por estos algoritmos ya que el pipeline no permite obtenerlo.

Además, aunque esto no aparezca, se ha realizado para cada una de las variables muestrales almacenadas en el apartado anterior.

In [ ]:
start = time.time()
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(x_train)
end = time.time()
print("Seconds: ", end - start)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
start = time.time()
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(x_train)
end = time.time()
print("Seconds: ", end - start)

### Entrenamiento de algoritmos no distribuidos

#### Multinominal NB
Para este entrenamiento se hace uso de un pre-procesamiento de la información mediante: CountVector + TF-IDF.  
Para su entrenamiento, se hace uso del Pipeline proporcionado por Scikit Learn que permite aplicar distintas funciones de manera secuencial, haciendo que el desarrollo sea más fácil de entender.  
Aunque este entrenamiento se ha realizado para cada uno de los tamaños muestrales definidos previamente, únicamente se muestra el entrenamiento con el 100% de los resultados.

In [ ]:
start = time.time()


pipeline = Pipeline([
    ('bow',CountVectorizer()), 
    ('tfidf',TfidfTransformer()), 
    ('classifier',MultinomialNB())
])

pipeline.fit(x_train, y_train)
end = time.time()
y_pred = pipeline.predict(x_test)
print('Accuracy: %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

print("Seconds: ", end - start)


#### Logistic Regression
Para este entrenamiento se hace uso de un pre-procesamiento de la información mediante: CountVector + TF-IDF.  
Para su entrenamiento, se hace uso del Pipeline proporcionado por Scikit Learn que permite aplicar distintas funciones de manera secuencial, haciendo que el desarrollo sea más fácil de entender.  
Aunque este entrenamiento se ha realizado para cada uno de los tamaños muestrales definidos previamente, únicamente se muestra el entrenamiento con el 100% de los resultados.

In [ ]:
from sklearn.linear_model import LogisticRegression

start = time.time()


logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression()),
               ])
logreg.fit(x_train, y_train)

end = time.time()

y_pred = logreg.predict(x_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

print("Seconds: ", end - start)

#### Random Forest

Para este entrenamiento se hace uso de un pre-procesamiento de la información mediante: CountVector + TF-IDF.  
Para su entrenamiento, se hace uso del Pipeline proporcionado por Scikit Learn que permite aplicar distintas funciones de manera secuencial, haciendo que el desarrollo sea más fácil de entender.  
Aunque este entrenamiento se ha realizado para cada uno de los tamaños muestrales definidos previamente, únicamente se muestra el entrenamiento con el 100% de los resultados.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
start = time.time()
pipeline = Pipeline([('bow', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('classifier', RandomForestClassifier(n_estimators=20)),
               ])
pipeline.fit(x_train, y_train)
end = time.time()
y_pred = pipeline.predict(x_test)
print('Accuracy: %s' % accuracy_score(y_pred, y_test))
print("Seconds: ", end - start)
print("-------------------------------------------")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
